## Study context

During the municipal election campaign, a party seeks to know how many documents per electoral district it has to prepare per polling station. This is to optimize the preparation for distribution by parcel carriers. The city concerned is Quimper.


In [1]:
import json
import geopy.geocoders
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
geopy.geocoders.options.default_user_agent = 'my_app/1'
import folium 
import pandas as pd
import requests



In [4]:
# data about the geographical perimeters of electoral offices

url_borough = 'https://www.data.gouv.fr/fr/datasets/r/2d84335f-1441-458e-9a7d-e356b81192aa'
elec_office= requests.get(url_borough).json()

In [5]:
#coordinates of Quimper

address = 'Quimper'

geolocator = Nominatim()
locationq = geolocator.geocode(address)
latitudeq = locationq.latitude
longitudeq = locationq.longitude
print('The geograpical coordinate of Quimper City are {}, {}.'.format(latitudeq, longitudeq))



The geograpical coordinate of Quimper City are 47.9960325, -4.1024782.


In [8]:
# show Quimper and its several electoral offices
quimper_map = folium.Map(location=[latitudeq,longitudeq ], zoom_start=12)
quimper_map .add_child(folium.GeoJson(elec_office))

quimper_map

In [23]:
# data about all adress of Quimper
adress_url = 'https://www.data.gouv.fr/fr/datasets/r/5ff3c371-d1bd-4b5e-83e5-50323dd0ca56'
adress= requests.get(adress_url).json()['features']

In [24]:
liste_adress = []
liste = []
for a in adress : 
    #only Quimper's adresses
    if a['properties']['IDCOMM'] == '290232' : 
        liste.append(a['properties']['IDCOMM'])
        liste.append(a['properties']['ADR_OBS'])
        liste.append(a['geometry']['coordinates'])
        liste_adress.append(liste)
        liste =[]

df_adress = pd.DataFrame(liste_adress)
df_adress.columns=['Id_city','Adress','coordinate']
df_adress.head()

,Id_city,Adress,coordinate
0,290232,8 RUE AMIRAL JULIEN COSMAO,"[-4.097744878879205, 48.0073865140059]"
1,290232,15 ALLEE MATHIAS LE LOUET,"[-4.127051366570884, 48.00509412450984]"
2,290232,15 RUE DE LOCRONAN,"[-4.108609193861332, 47.99790696909011]"
3,290232,20 CHEMIN DE KERLIVIDIC,"[-4.110064788799618, 48.029439632908826]"
4,290232,91 CHEMIN DE KERLIVIDIC,"[-4.117161885805333, 48.02676951721479]"


In [25]:
df_adress.shape

(25548, 3)

In [26]:
# build dataframe with electoral offices
listeo = []
listeoffice = []
for b in elec_office['features'] : 
    listeo.append(b['properties']['ADRESSE'])
    listeo.append(b['properties']['LIEU_VOTE'])
    listeo.append(b['properties']['BUREAUX'])
    listeo.append(b['geometry']['coordinates'])
    listeoffice.append(listeo)
    listeo = []
df_office = pd.DataFrame(listeoffice)
df_office.columns = ['adress','name','electoral office','limit']
df_office.head()

,adress,name,electoral office,limit
0,53 rue des Cerisiers,Ecole Elementaire de Kervilien,13,"[[[[-4.118259889941478, 47.98039688374357], [-..."
1,107 route de Pont-l'Abbé,Immeuble communal de Penanguer,12,"[[[[-4.120908265173999, 47.98075363864825], [-..."
2,114 bis avenue de Kergoat Ar Lez,Ecole Primaire Kergoat al Lez,23,"[[[[-4.083332382946192, 47.991199413091614], [..."
3,6 place du Cosmos,Ecole Primaire Pauline Kergomard,26,"[[[[-4.071659439189485, 47.993605349581706], [..."
4,14 rue des Trois Le Guennec,Ecole Primaire de Penanguer,11,"[[[[-4.118587205153393, 47.98975127084124], [-..."


In [27]:
# sort by number of electoral office
df_office = df_office.sort_values(by='electoral office')
df_office = df_office.reset_index()
df_office = df_office.drop(['index'],axis=1)
df_office.head()

,adress,name,electoral office,limit
0,18 bis rue Vis,Ecole Primaire Ferdinand Buisson,1,"[[[[-4.108673293265551, 47.99501961609982], [-..."
1,18 bis rue Vis,Ecole Primaire Ferdinand Buisson,2,"[[[[-4.112517294922924, 47.992208843904606], [..."
2,47 chemin de Prateyer,Maison de quartier du Moulin Vert,3,"[[[[-4.110138603043166, 48.00086973406141], [-..."
3,47 rue du Moulin Vert,Ecole Primaire Léon Goraguer,4,"[[[[-4.105270014879253, 48.00233931160333], [-..."
4,47 rue du Moulin Vert,Ecole Primaire Léon Goraguer,5,"[[[[-4.140739902820359, 48.01938027552455], [-..."


In [184]:
pip install shapely

     |████████████████████████████████| 1.6 MB 1.7 MB/s eta 0:00:01     |████▏                           | 215 kB 1.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [28]:
from shapely.geometry import Point, Polygon

In [29]:
# build dataframe with the center of electoral office perimeter to show on the map
centre = []
for p in range(df_office.shape[0]) :
            coords = df_office.loc[p,'limit'][0][0]
            poly = Polygon(coords)
            c = poly.centroid.coords
            centre.append(c)
df_centre = pd.DataFrame(centre)
df_centre.head()
df_centre=pd.concat([df_centre,df_office[['name','electoral office']]],axis=1)
df_centre.columns=['coordinate centre','name','electoral office']

In [30]:
df_centre.head()

,coordinate centre,name,electoral office
0,"(-4.1123023310345035, 47.99162418855593)",Ecole Primaire Ferdinand Buisson,1
1,"(-4.114556622282839, 47.99406642233797)",Ecole Primaire Ferdinand Buisson,2
2,"(-4.115403671361115, 47.999232833967355)",Maison de quartier du Moulin Vert,3
3,"(-4.111523345415905, 48.00443139438004)",Ecole Primaire Léon Goraguer,4
4,"(-4.129110175001721, 48.01080187532915)",Ecole Primaire Léon Goraguer,5


In [31]:
# show the name of electoral offices
for coordinate,name, nb_office in zip(df_centre['coordinate centre'],
                             df_centre['name'],df_centre['electoral office']):
        lat = coordinate[1]
        lng = coordinate[0]
        label = folium.Popup(str(nb_office) + '-' + name, parse_html = True)
        folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.6,
        parse_html = False).add_to(quimper_map) 
quimper_map    

In [34]:
# function to assign each adress to a electoral office
def office():
    liste = []
    resume = []
    compte=0
    for j in range(df_adress.shape[0]):
        p = Point(df_adress.loc[j,'coordinate'])

# detection in a Polygon
        for u in range(df_office.shape[0]) :
            coords = df_office.loc[u,'limit'][0][0]
            poly = Polygon(coords)
            if p.within(poly) == True: 
                liste.append(df_adress.loc[j,'Adress'])
                liste.append(df_adress.loc[j,'coordinate'])
                liste.append(df_office.loc[u,'electoral office'])
                resume.append(liste)
                liste = []
                break
    df_liste=pd.DataFrame(resume)
    df_liste.columns=['adress','coordinate','electoral office']
   
    return df_liste

In [35]:
# build the dataframe with the function office
df_adress_office = office()

In [36]:
df_adress_office.head()

,adress,coordinate,electoral office
0,8 RUE AMIRAL JULIEN COSMAO,"[-4.097744878879205, 48.0073865140059]",41
1,15 ALLEE MATHIAS LE LOUET,"[-4.127051366570884, 48.00509412450984]",6
2,15 RUE DE LOCRONAN,"[-4.108609193861332, 47.99790696909011]",40
3,20 CHEMIN DE KERLIVIDIC,"[-4.110064788799618, 48.029439632908826]",43
4,91 CHEMIN DE KERLIVIDIC,"[-4.117161885805333, 48.02676951721479]",43


In [81]:
# calculation of adresses numbers per electoral offices
office_count=df_adress_office.groupby('electoral office').count()

In [82]:
# clean the dataframe
office_count = office_count['adress']
office_count = office_count.reset_index()
office_count = office_count.rename(columns={'adress':'nb house'})
office_count ['electoral office'] = office_count['electoral office'].astype('int64')

In [83]:
# merge with the dataframe df_office 
office_count = pd.merge(office_count,df_office[['electoral office','adress','name']], on ='electoral office')
office_count.head()

,electoral office,nb house,adress,name
0,1,453,18 bis rue Vis,Ecole Primaire Ferdinand Buisson
1,2,298,18 bis rue Vis,Ecole Primaire Ferdinand Buisson
2,3,510,47 chemin de Prateyer,Maison de quartier du Moulin Vert
3,4,382,47 rue du Moulin Vert,Ecole Primaire Léon Goraguer
4,5,564,47 rue du Moulin Vert,Ecole Primaire Léon Goraguer


In [49]:
# test of a electoral office : 

df_adress_office49 = df_adress_office[df_adress_office['electoral office'] == 49]

In [50]:
# show the adresses classified to the electoral office 2 
quimper_map = folium.Map(location=[latitudeq,longitudeq ], zoom_start=12)
quimper_map .add_child(folium.GeoJson(elec_office))
for coordinate,name, nb_office in zip(df_centre['coordinate centre'],
                             df_centre['name'],df_centre['electoral office']):
        lat = coordinate[1]
        lng = coordinate[0]
        label = folium.Popup(str(nb_office) + '-' + name, parse_html = True)
        folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.6,
        parse_html = False).add_to(quimper_map) 
    
for coordinate, label in zip(df_adress_office49['coordinate'],
                             df_adress_office49['adress']):
        lat = coordinate[1]
        lng = coordinate[0]
        label = folium.Popup(label, parse_html = True)
        folium.CircleMarker(
        [lat, lng],
        radius = 1,
        popup = label,
        color = 'pink',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.6,
        parse_html = False).add_to(quimper_map) 
quimper_map    

We observe only pink markers in an electoral office perimeter. When we check the electoral office marker, it is the electoral office 49. 

Calculation hypothesis : 

sources :
* http://reseaux-chaleur.cerema.fr/wp-content/uploads/2008_Extrait_etude_CDC_Boucle_locale_optique___donnees_sur_le_parc_immobilier_francais.pdf

* http://www.journaldunet.com/management/ville/quimper/ville-29232/immobilier

47% of Quimper's dwellings are houses, 53% are apartments.    
in 2009 Quimper had 35984 dwellings, there is a growth of 4000 dwellings every 10 years.  
85% of buildings have 6 appartments, 13% of buildings have 16 appartments, 2% of buildings have 32 appartments.
  


In [54]:
# estimated number of dwellings in 2020
dwelling=round(35984+4000*11/10)
print('estimated number of dwellings in 2020: ',dwelling )

# estimated number of adresses representing houses : 
hose =round(dwelling*0.47)
print('houses en 2020: ',house)

# houses percentage per adress : 

houseperc = house/df_adress.shape[0]
print('houses percentage per adress:', houseperc )

#estimated number of adresses representing building : 
building = df_adress.shape[0] - house
print('buildings en 2020 : ',building)

# average appartment numbers per building

appart=round((dwelling-house)/building)
print("average appartment numbers per building:" ,appart)

estimated number of dwellings in 2020:  40384
houses en 2020:  18980
houses percentage per adress: 0.7429152966964145
buildings en 2020 :  6568
average appartment numbers per building: 3


When we apply the repartition of buildings with the number of appartments, we get to many dwellings.
So we choose to adapt the repartition like this : 
98% of buildings have 3 appartments, 2% of them have 16 appartments.

In [84]:
# calculation of numbers of mailboxes
office_count['nb house'] = round(office_count['nb house']*(0.74+0.26*(0.98*3+0.02*16)))
office_count = office_count.rename(columns={'nb house':'nb mailboxes'})

In [85]:
office_count

,electoral office,nb mailboxes,adress,name
0,1,719.0,18 bis rue Vis,Ecole Primaire Ferdinand Buisson
1,2,473.0,18 bis rue Vis,Ecole Primaire Ferdinand Buisson
2,3,810.0,47 chemin de Prateyer,Maison de quartier du Moulin Vert
3,4,606.0,47 rue du Moulin Vert,Ecole Primaire Léon Goraguer
4,5,895.0,47 rue du Moulin Vert,Ecole Primaire Léon Goraguer
5,6,1014.0,47 rue du Moulin Vert,Ecole Primaire Léon Goraguer
6,7,891.0,75 bis rue de la Terre Noire,Ecole Elémentaire Yves Le Manchec
7,8,1041.0,75 bis rue de la Terre Noire,Ecole Elémentaire Yves Le Manchec
8,9,643.0,2 place de Penhars,Ecole Maternelle du bourg de Penhars
9,10,695.0,2 place de Penhars,Ecole Maternelle du bourg de Penhars


In [88]:
# we check that our numbers of mailboxes has the same order of magnitude as the estimated number of dwellings
office_count['nb mailboxes'] .sum()

40544.0

In [89]:
# export the data as excel
office_count.to_excel('nbreparbureau.xlsx', engine='xlsxwriter')

## Results and conclusion

From geographical data on the perimeters of the various electoral offices and all the addresses in Quimper, we were able to determine the number of existing addresses per electoral office. 
The purpose of this study is to define the number of ballots needed for the electoral cocanditates. It was therefore necessary to estimate the number of houses and buildings. From the number of addresses we were able to give a first estimate of the number of mailboxes per electoral office. 

Of course, the basic data (percentage of houses, number of dwellings, etc.) used are average estimates. It is very likely that the distribution of houses and buildings is not the same in each electoral office perimeter. Similarly, the size of the buildings and therefore the number of apartments per building may vary. This is therefore an initial estimate of the need for ballot papers. These figures will have to be refined subsequently by field experience or a new source of data not currently available. 
